# Part I: Foundations of FastAPI

## Chapter 2: Routing and Basic Path Operations

Routing is the heart of any web API—it determines how your application responds to client requests for different URLs and HTTP methods. FastAPI's routing system is both powerful and intuitive, leveraging Python decorators and type hints to create a clean, declarative API structure. This chapter will teach you everything you need to know about defining routes, handling parameters, and processing request data.

---

### 2.1 The Path Operation Decorator

In FastAPI, a **path operation** is the combination of:
1. A **path** (URL route like `/users` or `/items/{id}`)
2. An **HTTP method** (GET, POST, PUT, DELETE, PATCH, etc.)
3. A **path operation function** that handles the request

The decorator syntax `@app.get()`, `@app.post()`, etc., is what registers these operations with your FastAPI application.

#### Understanding HTTP Methods

Before diving into decorators, let's review the standard HTTP methods and their conventional uses:

| Method | Purpose | Request Body | Response Body | Idempotent |
|--------|---------|--------------|---------------|------------|
| **GET** | Retrieve a resource | No | Yes | Yes |
| **POST** | Create a new resource | Yes | Yes | No |
| **PUT** | Replace an entire resource | Yes | Yes | Yes |
| **PATCH** | Partially update a resource | Yes | Yes | No |
| **DELETE** | Remove a resource | No | Yes | Yes |
| **HEAD** | Get headers only (no body) | No | No | Yes |
| **OPTIONS** | Get supported methods | No | Yes | Yes |

**Idempotent** means making the same request multiple times produces the same result on the server state.

---

> **RESTful Principle:** In REST API design, HTTP methods have semantic meaning. Using them according to their purpose (GET for reading, POST for creating, PUT for replacing, PATCH for updating, DELETE for removing) creates predictable, intuitive APIs.

---

#### The Decorator Syntax

Let's examine the complete decorator syntax:

```python
from fastapi import FastAPI

app = FastAPI()


@app.get(
    path="/users",
    response_model=None,
    status_code=200,
    tags=["users"],
    summary="Get all users",
    description="Retrieve a list of all users in the system",
    response_description="List of user objects",
    deprecated=False,
)
async def get_users():
    return [{"id": 1, "name": "Alice"}, {"id": 2, "name": "Bob"}]
```

The decorator accepts several parameters to customize the endpoint:

| Parameter | Type | Purpose |
|-----------|------|---------|
| `path` | `str` | The URL path (first positional argument) |
| `response_model` | `type` | Pydantic model for response validation |
| `status_code` | `int` | Default HTTP status code for responses |
| `tags` | `list[str]` | Groups endpoints in documentation |
| `summary` | `str` | Short description for the endpoint |
| `description` | `str` | Detailed explanation (supports Markdown) |
| `response_description` | `str` | Description of the response |
| `deprecated` | `bool` | Marks endpoint as deprecated |
| `name` | `str` | Custom operation ID (used for OpenAPI) |

In practice, most of these are optional, and FastAPI generates sensible defaults from your function:

```python
# Minimal version - FastAPI derives documentation from the code
@app.get("/users")
async def get_users():
    """Retrieve all users from the system."""  # This becomes the description
    return [{"id": 1, "name": "Alice"}]
```

#### Multiple HTTP Methods

You can register different functions for the same path with different methods:

```python
# main.py
from fastapi import FastAPI

app = FastAPI()


@app.get("/items")
async def list_items():
    """Retrieve all items."""
    return {"action": "list", "items": []}


@app.post("/items")
async def create_item():
    """Create a new item."""
    return {"action": "create", "id": 1}


@app.get("/items/{item_id}")
async def get_item(item_id: int):
    """Retrieve a specific item by ID."""
    return {"action": "get", "item_id": item_id}


@app.put("/items/{item_id}")
async def replace_item(item_id: int):
    """Replace an entire item."""
    return {"action": "replace", "item_id": item_id}


@app.patch("/items/{item_id}")
async def update_item(item_id: int):
    """Partially update an item."""
    return {"action": "update", "item_id": item_id}


@app.delete("/items/{item_id}")
async def delete_item(item_id: int):
    """Delete an item."""
    return {"action": "delete", "item_id": item_id}
```

Run this with `fastapi dev main.py` and explore the Swagger UI. You'll see all six operations grouped under the `/items` paths.

#### Combining Multiple Methods

For APIs that need to handle multiple HTTP methods with the same function, you can use `@app.api_route`:

```python
@app.api_route("/combined", methods=["GET", "POST"])
async def combined_handler(request: Request):
    if request.method == "GET":
        return {"method": "GET"}
    else:
        return {"method": "POST"}
```

---

> **Industry Standard:** Avoid combining methods unless you have a specific reason. Separate handlers for each method make your API more explicit and easier to understand.

---

### 2.2 Path Parameters

**Path parameters** are variable parts of a URL path. They are used to capture values from the URL itself, making them ideal for identifying specific resources.

#### Basic Path Parameters

Define a path parameter using curly braces `{}`:

```python
@app.get("/users/{user_id}")
async def get_user(user_id: int):
    """
    Retrieve a user by their unique identifier.

    - **user_id**: The integer ID of the user to retrieve
    """
    return {"user_id": user_id}
```

When a request is made to `/users/123`, FastAPI:
1. Matches the path `/users/123` to the pattern `/users/{user_id}`
2. Extracts the value `123` for the parameter `user_id`
3. Converts it to an integer (based on the type annotation `int`)
4. Passes it to your function as `user_id=123`

#### Type Conversion and Validation

FastAPI automatically converts path parameters to the annotated type:

```python
@app.get("/items/{item_id}")
async def read_item(item_id: int):
    # item_id is already an integer, not a string
    return {"item_id": item_id, "type": type(item_id).__name__}
```

Test with different inputs:

| URL | Response | Explanation |
|-----|----------|-------------|
| `/items/42` | `{"item_id": 42, "type": "int"}` | Valid integer |
| `/items/abc` | **422 Unprocessable Entity** | Cannot convert "abc" to int |
| `/items/3.14` | **422 Unprocessable Entity** | Cannot convert float to int |

The automatic validation error response is detailed and helpful:

```json
{
  "detail": [
    {
      "type": "int_parsing",
      "loc": ["path", "item_id"],
      "msg": "Input should be a valid integer, unable to parse string as an integer",
      "input": "abc",
      "url": "https://errors.pydantic.dev/2.5/v/int_parsing"
    }
  ]
}
```

#### Supported Path Parameter Types

FastAPI supports various types for path parameters:

```python
from uuid import UUID
from datetime import date


@app.get("/products/{product_id}")
async def get_product(product_id: int):
    return {"product_id": product_id, "type": "integer"}


@app.get("/orders/{order_id}")
async def get_order(order_id: UUID):
    return {"order_id": str(order_id), "type": "UUID"}


@app.get("/events/{event_date}")
async def get_event(event_date: date):
    return {
        "event_date": event_date.isoformat(),
        "type": "date",
        "year": event_date.year,
    }


@app.get("/files/{file_path:path}")
async def read_file(file_path: str):
    """
    The :path converter allows forward slashes in the parameter.
    """
    return {"file_path": file_path}
```

The special `:path` converter is worth noting—it allows the parameter to contain forward slashes:

```python
# Without :path converter
# URL: /files/docs/readme.txt -> file_path = "docs/readme.txt" ❌ (wouldn't match)

# With :path converter
# URL: /files/docs/readme.txt -> file_path = "docs/readme.txt" ✓
```

#### Path Parameter Validation with `Path`

For more control over validation, use the `Path` function from `fastapi`:

```python
from fastapi import FastAPI, Path

app = FastAPI()


@app.get("/products/{product_id}")
async def get_product(
    product_id: int = Path(
        gt=0,  # greater than 0
        le=1000,  # less than or equal to 1000
        description="The product ID must be between 1 and 1000",
        example=123,
    ),
):
    return {"product_id": product_id}
```

**Validation Options for Numeric Types:**

| Parameter | Meaning |
|-----------|---------|
| `gt` | Greater than |
| `ge` | Greater than or equal |
| `lt` | Less than |
| `le` | Less than or equal |

**Validation Options for Strings:**

| Parameter | Meaning |
|-----------|---------|
| `min_length` | Minimum string length |
| `max_length` | Maximum string length |
| `pattern` | Regular expression pattern |

Example with string validation:

```python
@app.get("/users/{username}")
async def get_user_by_username(
    username: str = Path(
        min_length=3,
        max_length=20,
        pattern=r"^[a-zA-Z0-9_]+$",  # alphanumeric and underscore only
        description="Username: 3-20 alphanumeric characters or underscores",
    ),
):
    return {"username": username}
```

#### Multiple Path Parameters

You can have multiple path parameters in a single route:

```python
@app.get("/users/{user_id}/posts/{post_id}")
async def get_user_post(
    user_id: int = Path(ge=1, description="The user ID"),
    post_id: int = Path(ge=1, description="The post ID"),
):
    return {"user_id": user_id, "post_id": post_id}
```

Request: `/users/42/posts/101`

Response:
```json
{
  "user_id": 42,
  "post_id": 101
}
```

#### Order of Routes Matters

FastAPI matches routes in the order they're defined. This can cause issues when you have overlapping paths:

```python
# ❌ WRONG ORDER
@app.get("/users/me")
async def get_current_user():
    return {"message": "This is the current user"}


@app.get("/users/{user_id}")
async def get_user(user_id: int):
    return {"user_id": user_id}


# Request to /users/me would return {"user_id": "me"} with a validation error!
```

Instead, define specific paths before parameterized ones:

```python
# ✓ CORRECT ORDER
@app.get("/users/me")
async def get_current_user():
    return {"message": "This is the current user"}


@app.get("/users/{user_id}")
async def get_user(user_id: int):
    return {"user_id": user_id}


# Request to /users/me returns {"message": "This is the current user"}
# Request to /users/42 returns {"user_id": 42}
```

---

> **Industry Standard:** Always define specific, fixed paths before paths with parameters. FastAPI matches routes sequentially, so a parameterized route like `/{name}` could capture requests intended for a fixed route like `/me` if defined first.

---

### 2.3 Query Parameters

**Query parameters** are the key-value pairs that appear after the `?` in a URL. They're commonly used for filtering, sorting, and pagination.

#### Basic Query Parameters

Define query parameters as function parameters with default values:

```python
from typing import Annotated

from fastapi import FastAPI, Query

app = FastAPI()


@app.get("/items")
async def list_items(
    skip: int = 0,
    limit: int = 10,
):
    """
    List items with pagination.

    - **skip**: Number of items to skip (default: 0)
    - **limit**: Maximum number of items to return (default: 10)
    """
    items = [{"id": i, "name": f"Item {i}"} for i in range(1, 101)]
    return items[skip : skip + limit]
```

Request examples:
- `/items` → Uses defaults: `skip=0`, `limit=10`
- `/items?skip=20` → `skip=20`, `limit=10` (default)
- `/items?skip=20&limit=5` → `skip=20`, `limit=5`

#### Required vs Optional Query Parameters

Query parameters with no default value are **required**:

```python
@app.get("/search")
async def search(
    query: str,  # Required - no default value
    page: int = 1,  # Optional - has default
):
    return {"query": query, "page": page}
```

Request: `/search` → **422 Unprocessable Entity** (query is required)

Request: `/search?query=fastapi` → `{"query": "fastapi", "page": 1}`

To make a parameter optional with `None` as default:

```python
from typing import Annotated


@app.get("/products")
async def list_products(
    category: str | None = None,  # Optional, can be None
    min_price: float | None = None,
    max_price: float | None = None,
):
    filters = {}
    if category:
        filters["category"] = category
    if min_price is not None:
        filters["min_price"] = min_price
    if max_price is not None:
        filters["max_price"] = max_price

    return {"filters": filters, "products": []}
```

#### Query Parameter Validation with `Query`

Use the `Query` function for validation and metadata:

```python
from typing import Annotated

from fastapi import FastAPI, Query

app = FastAPI()


@app.get("/items")
async def list_items(
    q: Annotated[
        str | None,
        Query(
            min_length=3,
            max_length=50,
            pattern=r"^[a-zA-Z0-9\s]+$",
            description="Search query string",
        ),
    ] = None,
    skip: Annotated[int, Query(ge=0, description="Items to skip")] = 0,
    limit: Annotated[int, Query(gt=0, le=100, description="Max items")] = 10,
):
    return {"q": q, "skip": skip, "limit": limit}
```

**Understanding `Annotated`:**

The `Annotated` type from Python's `typing` module allows you to attach metadata to type hints. FastAPI uses this metadata for validation and documentation:

```python
# Old style (still supported)
def list_items(q: str | None = Query(min_length=3)):


# New style with Annotated (recommended)
def list_items(q: Annotated[str | None, Query(min_length=3)] = None):
```

---

> **Industry Standard:** Use `Annotated` for all parameter validations. It separates the type annotation (`str | None`) from the validation metadata (`Query(...)`), improving code clarity and enabling better tooling support.

---

#### Alias and Deprecated Parameters

Query parameters can have aliases (useful when the Python variable name doesn't match the expected parameter name) and can be marked as deprecated:

```python
@app.get("/products")
async def list_products(
    q: Annotated[
        str | None,
        Query(
            alias="search-query",  # URL: /products?search-query=laptop
            deprecated=True,  # Shows as deprecated in docs
            description="Use 'query' instead",
        ),
    ] = None,
    query: Annotated[str | None, Query(min_length=1)] = None,
):
    return {"q": q, "query": query}
```

#### Default Values as Examples

You can provide example values that appear in the documentation:

```python
@app.get("/users")
async def search_users(
    name: Annotated[
        str | None,
        Query(
            description="Filter by user name",
            examples=["Alice", "Bob"],
        ),
    ] = None,
    role: Annotated[
        str | None,
        Query(
            description="Filter by role",
            examples=["admin", "user", "guest"],
        ),
    ] = None,
):
    return {"name": name, "role": role}
```

### 2.4 Request Bodies

When clients need to send data to your API (e.g., creating or updating resources), they use the **request body**. FastAPI uses Pydantic models to define, validate, and document request bodies.

#### Introduction to Pydantic Models

**Pydantic** is a data validation library that uses Python type annotations. It's the foundation of FastAPI's request handling:

```python
from fastapi import FastAPI
from pydantic import BaseModel, Field

app = FastAPI()


class Item(BaseModel):
    """Model representing an item in our store."""

    name: str
    description: str | None = None
    price: float
    tax: float | None = None

    model_config = {
        "json_schema_extra": {
            "examples": [
                {
                    "name": "Laptop",
                    "description": "A powerful laptop",
                    "price": 999.99,
                    "tax": 0.1,
                }
            ]
        }
    }
```

Key features:
- Type validation: `price` must be a float
- Optional fields: `description` can be `None`
- Default values: Fields without defaults are required
- Documentation: The model becomes part of your OpenAPI schema

#### Creating a POST Endpoint

Use the Pydantic model as a type annotation in your path operation function:

```python
@app.post("/items")
async def create_item(item: Item):
    """
    Create a new item.

    Request body should be JSON matching the Item model.
    """
    # At this point, 'item' is a validated Pydantic model instance
    # FastAPI has already validated the incoming JSON
    result = item.model_dump()  # Convert to dict
    result.update({"total_price": item.price + (item.tax or 0)})
    return result
```

**How it works:**

1. Client sends POST request with JSON body
2. FastAPI parses the JSON
3. Pydantic validates data against your model
4. If valid, your function receives a proper `Item` instance
5. If invalid, FastAPI returns a detailed 422 error

**Example Request:**

```bash
curl -X POST http://localhost:8000/items \
  -H "Content-Type: application/json" \
  -d '{"name": "Laptop", "price": 999.99, "tax": 50.00}'
```

**Response:**

```json
{
  "name": "Laptop",
  "description": null,
  "price": 999.99,
  "tax": 50.0,
  "total_price": 1049.99
}
```

#### Multiple Body Parameters

Sometimes you need multiple objects in the request body:

```python
class User(BaseModel):
    username: str
    email: str


class Item(BaseModel):
    name: str
    price: float


@app.post("/items-with-owner")
async def create_item_with_owner(item: Item, user: User):
    """
    Both 'item' and 'user' are expected as JSON keys in the body.

    Expected body:
    {
        "item": {"name": "Laptop", "price": 999.99},
        "user": {"username": "alice", "email": "alice@example.com"}
    }
    """
    return {"item": item, "user": user}
```

#### Using `Body` for Singular Values

To receive a single value from the body (not a model), use `Body`:

```python
from fastapi import Body


@app.post("/items/{item_id}")
async def update_item(
    item_id: int,
    name: Annotated[str, Body(min_length=1)],
    price: Annotated[float, Body(gt=0)],
):
    return {"item_id": item_id, "name": name, "price": price}
```

Expected request body:
```json
{
  "name": "Updated Laptop",
  "price": 1099.99
}
```

#### Combining Path, Query, and Body Parameters

You can mix all parameter types in a single endpoint:

```python
from fastapi import FastAPI, Body

app = FastAPI()


class Item(BaseModel):
    name: str
    price: float
    is_offer: bool = False


@app.put("/items/{item_id}")
async def update_item(
    item_id: Annotated[int, Path(gt=0, description="The item ID")],
    q: Annotated[str | None, Query(description="Search query")] = None,
    item: Annotated[Item, Body(description="The item data to update")] = None,
):
    """
    Update an item.

    - **item_id**: Path parameter (required)
    - **q**: Optional query parameter
    - **item**: Request body as JSON
    """
    result = {"item_id": item_id}
    if q:
        result["q"] = q
    if item:
        result["item"] = item.model_dump()
    return result
```

Request example:
```bash
curl -X PUT "http://localhost:8000/items/42?q=urgent" \
  -H "Content-Type: application/json" \
  -d '{"name": "Gaming Laptop", "price": 1299.99, "is_offer": true}'
```

Response:
```json
{
  "item_id": 42,
  "q": "urgent",
  "item": {
    "name": "Gaming Laptop",
    "price": 1299.99,
    "is_offer": true
  }
}
```

### 2.5 Response Models

Response models define the structure of your API's output. They provide:
- **Validation**: Ensure responses match expected format
- **Filtering**: Exclude sensitive fields from output
- **Documentation**: Auto-generate OpenAPI response schemas

#### Basic Response Model

Use the `response_model` parameter in the decorator:

```python
from fastapi import FastAPI
from pydantic import BaseModel, Field

app = FastAPI()


class UserResponse(BaseModel):
    """User data returned by the API."""

    id: int
    username: str
    email: str
    is_active: bool = True


class UserInDB(BaseModel):
    """Complete user data in the database (includes sensitive fields)."""

    id: int
    username: str
    email: str
    hashed_password: str  # Sensitive - should NOT be in response
    is_active: bool
    created_at: str


# Simulated database
fake_db = {
    1: UserInDB(
        id=1,
        username="alice",
        email="alice@example.com",
        hashed_password="$2b$12$...",
        is_active=True,
        created_at="2024-01-15",
    )
}


@app.get("/users/{user_id}", response_model=UserResponse)
async def get_user(user_id: int):
    """
    Get user by ID.

    Even though the database contains a hashed_password,
    the response will only include fields defined in UserResponse.
    """
    user = fake_db.get(user_id)
    if user is None:
        from fastapi import HTTPException

        raise HTTPException(status_code=404, detail="User not found")
    return user
```

Request: `GET /users/1`

Response (password automatically filtered out):
```json
{
  "id": 1,
  "username": "alice",
  "email": "alice@example.com",
  "is_active": true
}
```

#### Response Model with Field Filtering

You can also control filtering within the model itself:

```python
from pydantic import BaseModel, Field


class User(BaseModel):
    id: int
    username: str
    email: str
    password: str = Field(exclude=True)  # Never include in output
    api_key: str = Field(exclude=True)


@app.get("/profile", response_model=User)
async def get_profile():
    return {
        "id": 1,
        "username": "alice",
        "email": "alice@example.com",
        "password": "secret123",  # Won't appear in response
        "api_key": "key-abc-123",  # Won't appear in response
    }
```

#### Multiple Response Models

Different endpoints can return different models for the same resource:

```python
class UserSummary(BaseModel):
    """Brief user info for listings."""

    id: int
    username: str


class UserDetail(BaseModel):
    """Complete user info for detail view."""

    id: int
    username: str
    email: str
    bio: str | None = None
    created_at: str


@app.get("/users", response_model=list[UserSummary])
async def list_users():
    """Return a list of users with summary info."""
    return [
        {"id": 1, "username": "alice"},
        {"id": 2, "username": "bob"},
        {"id": 3, "username": "charlie"},
    ]


@app.get("/users/{user_id}", response_model=UserDetail)
async def get_user(user_id: int):
    """Return detailed user information."""
    return {
        "id": user_id,
        "username": "alice",
        "email": "alice@example.com",
        "bio": "Python developer",
        "created_at": "2024-01-15",
    }
```

#### Response Model for Different Status Codes

You can define different response models for different status codes:

```python
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

app = FastAPI()


class SuccessResponse(BaseModel):
    success: bool = True
    data: dict


class ErrorResponse(BaseModel):
    success: bool = False
    error: str


@app.get(
    "/items/{item_id}",
    response_model=SuccessResponse,
    responses={
        404: {"model": ErrorResponse, "description": "Item not found"},
        400: {"model": ErrorResponse, "description": "Invalid item ID"},
    },
)
async def get_item(item_id: int):
    if item_id < 0:
        raise HTTPException(
            status_code=400,
            detail={"success": False, "error": "Item ID must be positive"},
        )
    if item_id > 100:
        raise HTTPException(
            status_code=404,
            detail={"success": False, "error": "Item not found"},
        )
    return {"success": True, "data": {"id": item_id, "name": f"Item {item_id}"}}
```

The documentation will show all possible response models:

![Response models in documentation](placeholder)

#### Setting the Response Status Code

Control the HTTP status code for responses:

```python
from fastapi import FastAPI, status

app = FastAPI()


@app.post("/items", status_code=status.HTTP_201_CREATED)
async def create_item(name: str):
    """
    Create a new item.

    Returns 201 Created instead of the default 200 OK.
    """
    return {"name": name, "id": 1}
```

Common status codes from `fastapi.status`:

| Constant | Code | When to Use |
|----------|------|-------------|
| `HTTP_200_OK` | 200 | Successful GET, PUT, PATCH |
| `HTTP_201_CREATED` | 201 | Successful POST (resource created) |
| `HTTP_204_NO_CONTENT` | 204 | Successful DELETE or empty response |
| `HTTP_400_BAD_REQUEST` | 400 | Invalid client input |
| `HTTP_401_UNAUTHORIZED` | 401 | Authentication required |
| `HTTP_403_FORBIDDEN` | 403 | Permission denied |
| `HTTP_404_NOT_FOUND` | 404 | Resource doesn't exist |
| `HTTP_422_UNPROCESSABLE_ENTITY` | 422 | Validation error (automatic) |
| `HTTP_500_INTERNAL_SERVER_ERROR` | 500 | Server error |

#### Return Type Annotation as Response Model

You can use the return type annotation directly as the response model:

```python
@app.get("/users/{user_id}")
async def get_user(user_id: int) -> UserResponse:
    """
    The return type annotation sets the response model automatically.
    """
    return {"id": user_id, "username": "alice", "email": "alice@example.com"}
```

This is equivalent to setting `response_model=UserResponse` in the decorator but keeps the model visible in the function signature.

---

### Summary

In this chapter, you've learned the fundamentals of routing in FastAPI:

1. **Path Operation Decorators**: How to use `@app.get`, `@app.post`, and other HTTP method decorators with various configuration options.

2. **Path Parameters**: Defining dynamic URL segments with type conversion, validation using `Path()`, and understanding route ordering.

3. **Query Parameters**: Handling optional parameters, using `Query()` for validation, and working with aliases and deprecated parameters.

4. **Request Bodies**: Creating Pydantic models for request validation, handling multiple body parameters, and combining all parameter types.

5. **Response Models**: Filtering output, documenting responses, and handling multiple response types for different status codes.

---

### Exercises

1. **CRUD Operations**: Create a complete set of CRUD endpoints for a `Book` model:
   - `GET /books` - List all books with pagination (`skip`, `limit` query params)
   - `POST /books` - Create a new book
   - `GET /books/{book_id}` - Get a specific book
   - `PUT /books/{book_id}` - Replace a book entirely
   - `DELETE /books/{book_id}` - Delete a book (return 204 No Content)

2. **Validation Challenge**: Create an endpoint `GET /search` that accepts:
   - `query` (required string, 1-100 characters)
   - `category` (optional string, must be one of: "books", "electronics", "clothing")
   - `min_price` and `max_price` (optional floats, min_price must be less than max_price)

3. **Response Filtering**: Create a `User` model with `id`, `username`, `email`, `password`, and `created_at`. Create two endpoints:
   - `GET /users` returning only `id` and `username`
   - `GET /users/{user_id}` returning everything except `password`

4. **Combined Parameters**: Create an endpoint `PATCH /articles/{article_id}` that accepts:
   - `article_id` as a path parameter (positive integer)
   - `author` as a query parameter (optional)
   - A request body with `title` and `content` fields
   - Return the updated article with a `200 OK` status

---

### What's Next?

**Chapter 3: Automatic Documentation** will explore FastAPI's built-in documentation capabilities:
- Deep dive into Swagger UI features and customization
- Configuring ReDoc for alternative documentation views
- Understanding the OpenAPI specification structure
- Organizing endpoints with tags and grouping
- Adding examples, descriptions, and deprecation notices
- Customizing the documentation URL paths

